## THIS IS THE CODE FOR Ambition Jewels consider PO: NL PO# 101536 AMBITION (SHIMAYRA)-EDITING.pdf

### same code for retriving data from pdf as you sent obu:

In [10]:
pip install pandas regex pdfplumber openpyxl


Defaulting to user installation because normal site-packages is not writeable
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd  # #PDF TO TEXT PO1
import re
import pdfplumber

def extract_data_from_pdf(pdf_path):
    """
    Extract data from PDF file and return structured data
    """
    data = []
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Extract text from all pages
            full_text = ""
            for page in pdf.pages:
                full_text += page.extract_text() + "\n"
            
            print("Extracted text from PDF:")
            print("=" * 50)
            print(full_text)
            print("=" * 50)
            
            # Process the extracted text
            lines = full_text.split('\n')
            current_item = {}
            
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                    
                # Extract SrNo (looking for lines starting with numbers)
                sr_no_match = re.match(r'^\s*(\d+)\s*', line)
                if sr_no_match:
                    if current_item and 'SrNo' in current_item:  # Save previous item if exists
                        data.append(current_item)
                    current_item = {'SrNo': sr_no_match.group(1), 'OrderQty': 2}
                    
                # Extract Article code (patterns like 9-DD106-YG-25, 9-DD106-YG-7)
                article_match = re.search(r'([A-Z0-9\-]+YG\-?\d*\.?\d+)', line)
                if article_match and 'ArticleCode' not in current_item:
                    current_item['ArticleCode'] = article_match.group(1)
                    
                # Extract StyleCode from Your reference (text after RSBR2074-01, RSBR2074-03, etc.)
                style_match = re.search(r'RSBR2074\-(\d+)\s+([A-Z0-9]+)', line)
                if style_match:
                    current_item['StyleCode'] = style_match.group(2)
                    
                # Extract ItemSize from description (looking for numbers like 0.25, 0.07)
                # Multiple patterns to catch different formats
                size_patterns = [
                    r'pendant\s+(\d+\.\d+)',  # pendant 0.25
                    r'(\d+\.\d+)\s*ct',       # 0.25 ct
                    r'YG[-\s]*(\d+\.\d+)',    # YG-0.25 or YG 0.25
                ]
                
                for pattern in size_patterns:
                    size_match = re.search(pattern, line, re.IGNORECASE)
                    if size_match and 'ItemSize' not in current_item:
                        current_item['ItemSize'] = size_match.group(1)
                        break
            
            # Add the last item
            if current_item and 'SrNo' in current_item:
                data.append(current_item)
                
    except Exception as e:
        print(f"Error reading PDF file: {e}")
        return []
    
    return data

def create_excel_dataframe(data):
    """
    Create DataFrame with specified columns
    """
    if not data:
        print("No data extracted from PDF")
        return pd.DataFrame()
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Reorder columns as requested
    columns_order = ['SrNo', 'StyleCode', 'ItemSize', 'OrderQty']
    
    # Only include columns that exist in the data
    final_columns = [col for col in columns_order if col in df.columns]
    
    # Add missing columns with empty values
    for col in columns_order:
        if col not in df.columns:
            df[col] = ""
    
    return df[columns_order]

# Main execution
if __name__ == "__main__":
    # Specify your PDF file path here
   
    pdf_file_path = r'C:\Users\Admin\Desktop\Aanya Inc\PO#6393.pdf'  # Change this to your actual PDF path
    
    # Extract data from PDF
    print(f"Reading PDF from: {pdf_file_path}")
    extracted_data = extract_data_from_pdf(pdf_file_path)
    
    if extracted_data:
        # Create DataFrame
        df = create_excel_dataframe(extracted_data)
        
        # Display the results
        print("\nExtracted Data:")
        print("=" * 50)
        print(df)
        
        # Save to Excel file
        output_file = 'extracted_data.xlsx'
        df.to_excel(output_file, index=False)
        print(f"\nData successfully saved to '{output_file}'")
        
        # Display basic statistics
        print(f"\nExtraction Summary:")
        print(f"Total records extracted: {len(df)}")
        print(f"Columns: {list(df.columns)}")
        
    else:
        print("No data was extracted from the PDF file.")
    
    # Display the DataFrame in notebook
    df

Reading PDF from: C:\Users\Admin\Desktop\Aanya Inc\PO#6393.pdf
Extracted text from PDF:
Aanya Inc
PO # 6393
48 W 48th Street, PO Date 08/05/2025
# 506
Ship Date 09/05/2025
New York, NY 10036
Gold Lock $3,300
Phone: 917-512-6933
Silver Lock $0.00
Email: cs@aanyainc.com
Terms
Purchase Order
Vendor Remarks
SHIMAYRA JEWELLERY 14K ,A
PLOT NO: 62, SEEPZ
ANDHERI (E), MUMBAI:400096,India
saurabh.dalal@jasanigroup.com
Sr
Item No Image Description Qty Price Amount
No
1.0 CT TW Round Shaped Diamond Band Size-7
1 QR0350H-I1/7 70.00 $0.00 $0.00
Total 70.00 $0.00
Notes
Need Hallmark & Trademark on every piece.
All PO's must be confirmed within 24-48 hours of receipt.
1/1


Extracted Data:
  SrNo StyleCode ItemSize  OrderQty
0   48                NaN         2
1    1                1.0         2
2    1                NaN         2
3    1                NaN         2

Data successfully saved to 'extracted_data.xlsx'

Extraction Summary:
Total records extracted: 4
Columns: ['SrNo', 'StyleCode', 'ItemSi

#logic for the AMBITION PO:

**PROMPT**:

consider the first cell in the @Ambition.ipynb file from which the text is extracted for the text extracted create a dataframe and arrange that text in the following columns in a dataframe and save that in a excel sheet, so add the cell accordingly at the end of the @Ambition.ipynb file to do so, the instructions for each columns to be filled are as follows,

COLUMNS:

for each item the details will start from the Sr.No. here the first item starts from "1." in the text and the data for that item lasts till  "**FOR SHIMAYRA**" is seen
1.SrNo: The first number that is present after the 5th line in the text for example here after the 5th line after the word "style" in next line "1." is the Sr.No. for this item similarly for the next ITem if at the end of the item after "**FOR SHIMAYRA**" is next Sr.No. if is not seen it then means that the file end there do not consider any thing after that.

2.StyleCode: the StyleCode for a item is the second last code in the Sr.No. line for example here the style code for first item is: 'ZNLSN00814' similarly find for other items

3.ItemSize:itemsize is the first decimal number that is present in the Sr.No. line for example here item size is "18.00"

4.OrderQty: the number that is present right after the ItemSize is the orderQty for example here order quantity is "38"

5.OrderItemPcs: leave this column blank

6.Metal: for different items there are difference Metal code for example for example if "SILV" is present in the text then it is AG925 so whenever "SILV" is encountered assign "AG925" as Metal for that item and similary "G14W" for Gold 14kt present in the text and "W" is the tone present which will depend on the tone column 

7.Tone: after the "SILV" word/letter is encountered for that item there will be the tone for example here "W" is the tone

8.ItemPoNo.: its the same for all the items here in the text at the top PO # : 101533 "101533" is the ItemPoNo

9.ItemRefNo: the first digits that are present just after the Sr.No. is the ItemRefNo for example here "101536/101" is the ItemRefNo for that item

10.StockType: leave it blank

11.MakeType: leave it blank

12.CustomerProductionInstruction: for example here "1/2 CT LCD BUTTERCUP ETERNIT NECK GSI CB" is the CustomerProductionInstruction for that item similarly find for other items too if present

13.SpecialRemarks: here "101536/101  LN2095SS-24 SILV  SZ-18inch,  DIA I SI2" is the SpecialRemarks for the item so find similarly where it is present in the text and map according to the items in the text.

14.DesignProductionInstruction: here "**GSI MIN CERT** **NO GSI# ENGRAVING ON PCS** **DIRECT SHIP TO CANADA**" is the DesignProductionInstruction for the item similarly find for other items too

15.StampInstruction: here "STAMP 925 LGD ANJ" is the StampInstruction similarly find for other items too

16.OrderGroup: leave it blank

17.Certificate: leave it blank

18.SKUNo: here "LN2095SS-24" is the SKUNo for the item similarly find for other items too

19.Basestoneminwt:leave it blank

20.Basestonemaxwt:leave it blank

21.Basemetalminwt:leave it blank

22.Basemetalmaxwt:leave it blank

23.Productiondeliverydate:leave it blank

24.Expecteddeliverydate:leave it blank

25. create a blank column:leave it blank

26.SetPrice:leave it blank

27.StoneQuality:leave it blank



In [3]:
# Data processing for PO # 6393 - Create DataFrame with all required columns
import pandas as pd
import re

def process_po_6393_data():
    """
    Process the extracted text from PO # 6393 and create DataFrame with specified columns
    """
    
    # The extracted text from the PDF
    extracted_text = """Aanya Inc
PO # 6393
48 W 48th Street, PO Date 08/05/2025
# 506
Ship Date 09/05/2025
New York, NY 10036
Gold Lock $3,300
Phone: 917-512-6933
Silver Lock $0.00
Email: cs@aanyainc.com
Terms
Purchase Order
Vendor Remarks
SHIMAYRA JEWELLERY 14K ,A
PLOT NO: 62, SEEPZ
ANDHERI (E), MUMBAI:400096,India
saurabh.dalal@jasanigroup.com
Sr
Item No Image Description Qty Price Amount
No
1.0 CT TW Round Shaped Diamond Band Size-7
1 QR0350H-I1/7 70.00 $0.00 $0.00
Total 70.00 $0.00
Notes
Need Hallmark & Trademark on every piece.
All PO's must be confirmed within 24-48 hours of receipt.
1/1"""
    
    # Initialize the data list
    data = []
    
    # Extract information using regex patterns
    lines = extracted_text.split('\n')
    
    # Find PO number
    po_match = re.search(r'PO # (\d+)', extracted_text)
    po_number = po_match.group(1) if po_match else ""
    
    # Find metal type
    metal_match = re.search(r'SHIMAYRA JEWELLERY (\w+)', extracted_text)
    metal = metal_match.group(1) if metal_match else ""
    
    # Find the item description and details
    # Looking for pattern: "1.0 CT TW Round Shaped Diamond Band Size-7"
    item_desc_match = re.search(r'(\d+\.\d+ CT TW [^\n]+Size-(\d+))', extracted_text)
    if item_desc_match:
        customer_instruction = item_desc_match.group(1).replace(f"Size-{item_desc_match.group(2)}", f"SZ-{item_desc_match.group(2)}")
        item_size = f"UP{item_desc_match.group(2).zfill(2)}"  # zfill(2) ensures two digits like UP07
    
    # Find StyleCode and OrderQty
    style_qty_match = re.search(r'(\d+)\s+([A-Z0-9]+-[A-Z0-9/]+)\s+(\d+\.\d+)', extracted_text)
    if style_qty_match:
        sr_no = style_qty_match.group(1)
        style_code = style_qty_match.group(2)
        order_qty = style_qty_match.group(3).split('.')[0]  # Remove decimal part
    
    # Find special remarks
    special_remarks_match = re.search(r'Need Hallmark & Trademark on every piece\.', extracted_text)
    special_remarks = special_remarks_match.group(0) if special_remarks_match else ""
    
    # Debug output to verify extraction
    print("Debug - Extracted values:")
    print(f"SrNo: {sr_no if 'sr_no' in locals() else 'NOT FOUND'}")
    print(f"StyleCode: {style_code if 'style_code' in locals() else 'NOT FOUND'}")
    print(f"ItemSize: {item_size if 'item_size' in locals() else 'NOT FOUND'}")
    print(f"OrderQty: {order_qty if 'order_qty' in locals() else 'NOT FOUND'}")
    print(f"CustomerProductionInstruction: {customer_instruction if 'customer_instruction' in locals() else 'NOT FOUND'}")
    print(f"Metal: {metal}")
    print(f"ItemPoNo: {po_number}")
    print(f"SpecialRemarks: {special_remarks}")
    print("-" * 50)
    
    # Create the data dictionary with all required columns
    item_data = {
        'SrNo': sr_no if 'sr_no' in locals() else "",
        'StyleCode': style_code if 'style_code' in locals() else "",
        'ItemSize': item_size if 'item_size' in locals() else "",
        'OrderQty': order_qty if 'order_qty' in locals() else "",
        'OrderItemPcs': "",  # Leave blank
        'Metal': metal,
        'Tone': "",  # Leave blank
        'ItemPoNo.': po_number,
        'ItemRefNo': "",  # Leave blank
        'StockType': "",  # Leave blank
        'MakeType': "",  # Leave blank
        'CustomerProductionInstruction': customer_instruction if 'customer_instruction' in locals() else "",
        'SpecialRemarks': special_remarks,
        'DesignProductionInstruction': "",  # Leave blank
        'StampInstruction': "'A' on one side and metal KT on other side of the ring",  # Default value
        'OrderGroup': "",  # Leave blank
        'Certificate': "",  # Leave blank
        'SKUNo': "",  # Leave blank
        'Basestoneminwt': "",  # Leave blank
        'Basestonemaxwt': "",  # Leave blank
        'Basemetalminwt': "",  # Leave blank
        'Basemetalmaxwt': "",  # Leave blank
        'Productiondeliverydate': "",  # Leave blank
        'Expecteddeliverydate': "",  # Leave blank
        'Blank_Column': "",  # Leave blank
        'SetPrice': "",  # Leave blank
        'StoneQuality': ""  # Leave blank
    }
    
    data.append(item_data)
    
    return data

# Process the data
processed_data = process_po_6393_data()

# Create DataFrame
df = pd.DataFrame(processed_data)

# Display the DataFrame
print("Processed DataFrame:")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

# Save to Excel file
output_filename = 'PO_6393_extracted_data1.xlsx'
df.to_excel(output_filename, index=False)

print(f"\nData successfully saved to '{output_filename}'")
print(f"Total items processed: {len(df)}")
print(f"Columns created: {len(df.columns)}")

# Display column names
print(f"\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

df


Debug - Extracted values:
SrNo: 1
StyleCode: QR0350H-I1/7
ItemSize: UP07
OrderQty: 70
CustomerProductionInstruction: 1.0 CT TW Round Shaped Diamond Band SZ-7
Metal: 14K
ItemPoNo: 6393
SpecialRemarks: Need Hallmark & Trademark on every piece.
--------------------------------------------------
Processed DataFrame:
SrNo    StyleCode ItemSize OrderQty OrderItemPcs Metal Tone ItemPoNo. ItemRefNo StockType MakeType            CustomerProductionInstruction                            SpecialRemarks DesignProductionInstruction                                       StampInstruction OrderGroup Certificate SKUNo Basestoneminwt Basestonemaxwt Basemetalminwt Basemetalmaxwt Productiondeliverydate Expecteddeliverydate Blank_Column SetPrice StoneQuality
   1 QR0350H-I1/7     UP07       70                14K           6393                              1.0 CT TW Round Shaped Diamond Band SZ-7 Need Hallmark & Trademark on every piece.                             'A' on one side and metal KT on other side 

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo.,ItemRefNo,StockType,...,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,Blank_Column,SetPrice,StoneQuality
0,1,QR0350H-I1/7,UP07,70,,14K,,6393,,,...,,,,,,,,,,


In [12]:
pip install xlrd

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Excel file processing with mapping logic
import pandas as pd
import re

def process_excel_data(excel_file_path):
    """
    Process Excel file data and create DataFrame with specified columns
    """
    
    try:
        # Read the Excel file
        df = pd.read_excel(excel_file_path)
        
        print("Original Excel file columns:")
        print(df.columns.tolist())
        print("\nFirst few rows:")
        print(df.head())
        print("\n" + "="*80)
        
        # Get PO number from G5 cell (assuming it's in a specific sheet or cell)
        # If it's in a separate cell, we might need to read it differently
        po_number = ""  # Will be extracted from G5 cell
        
        # Initialize the data list
        data = []
        
        # Process each row in the dataframe
        for index, row in df.iterrows():
            # Initialize variables
            sr_no = ""
            style_code = ""
            item_size = ""
            order_qty = ""
            customer_instruction = ""
            special_remarks = ""
            metal = ""
            tone = ""
            
            # 1) Serial No as it is from the df
            if 'Serial No' in df.columns:
                sr_no = str(row['Serial No']) if pd.notna(row['Serial No']) else ""
            
            # 2) "StyleNo" is "StyleCode" for that particular item
            if 'StyleNo' in df.columns:
                style_code = str(row['StyleNo']) if pd.notna(row['StyleNo']) else ""
            
            # 3) ItemSize logic - map according to Sizes column from df
            if 'Sizes' in df.columns:
                size_value = str(row['Sizes']) if pd.notna(row['Sizes']) else ""
                # Extract number from size (e.g., "7" from "Size-7" or just "7")
                size_match = re.search(r'(\d+)', size_value)
                if size_match:
                    size_num = size_match.group(1)
                    item_size = f"UP{size_num.zfill(2)}"  # Format as UP07, UP10, etc.
            
            # 4) "OrderQty" column is "Qty" column
            if 'Qty' in df.columns:
                qty_value = row['Qty']
                if pd.notna(qty_value):
                    # Remove decimal part if present
                    order_qty = str(int(qty_value)) if isinstance(qty_value, (int, float)) else str(qty_value)
            
            # 5) "CustomerProductionInstruction" column map according to "Diamonds" column from df
            if 'Diamonds' in df.columns:
                diamonds_value = str(row['Diamonds']) if pd.notna(row['Diamonds']) else ""
                # Replace "Size-" with "SZ-" in the diamonds description
                customer_instruction = re.sub(r'Size-(\d+)', r'SZ-\1', diamonds_value)
            
            # 6) SpecialRemarks - where Serial No ends, next row's 1st cell is SpecialRemarks
            # This logic needs to be implemented based on how the data is structured
            if index + 1 < len(df):
                next_row = df.iloc[index + 1]
                # Assuming the first column contains the special remarks for the next item
                first_col_name = df.columns[0]
                special_remarks = str(next_row[first_col_name]) if pd.notna(next_row[first_col_name]) else ""
            
            # 7) Metal column from Description column
            if 'Description' in df.columns:
                description = str(row['Description']) if pd.notna(row['Description']) else ""
                
                # Extract metal information from description
                # Pattern for "14KT \nWhite Gold\nMin-4.6gm" -> "G14W"
                # Pattern for "18KT \nYellow Gold\nMin-4.6gm" -> "G18Y"
                
                # Remove \n characters first
                description_clean = description.replace('\n', ' ')
                
                # Look for patterns like "14KT White Gold" or "18KT Yellow Gold"
                metal_patterns = [
                    r'(\d+)KT\s+White\s+Gold',  # 14KT White Gold -> G14W
                    r'(\d+)KT\s+Yellow\s+Gold', # 18KT Yellow Gold -> G18Y
                    r'(\d+)KT\s+Rose\s+Gold',   # 14KT Rose Gold -> G14R
                    r'(\d+)KT\s+Gold',          # Generic gold -> G14 (assume 14K)
                ]
                
                for pattern in metal_patterns:
                    match = re.search(pattern, description_clean, re.IGNORECASE)
                    if match:
                        kt_value = match.group(1)
                        if 'white' in match.group(0).lower():
                            metal = f"G{kt_value}W"
                        elif 'yellow' in match.group(0).lower():
                            metal = f"G{kt_value}Y"
                        elif 'rose' in match.group(0).lower():
                            metal = f"G{kt_value}R"
                        else:
                            metal = f"G{kt_value}"  # Generic gold
                        break
            
            # 8) Tone column from Metal column
            if metal:
                # Extract tone from metal code (last character)
                tone = metal[-1] if len(metal) > 1 and metal[-1].isalpha() else ""
            
            # Create the data dictionary with all required columns
            item_data = {
                'SrNo': sr_no,
                'StyleCode': style_code,
                'ItemSize': item_size,
                'OrderQty': order_qty,
                'OrderItemPcs': "",  # Leave blank
                'Metal': metal,
                'Tone': tone,
                'ItemPoNo.': po_number,  # Will be extracted from G5 cell
                'ItemRefNo': "",  # Leave blank
                'StockType': "",  # Leave blank
                'MakeType': "",  # Leave blank
                'CustomerProductionInstruction': customer_instruction,
                'SpecialRemarks': special_remarks,
                'DesignProductionInstruction': "",  # Leave blank
                'StampInstruction': "'A' on one side and metal KT on other side of the ring",  # Default value
                'OrderGroup': "",  # Leave blank
                'Certificate': "",  # Leave blank
                'SKUNo': "",  # Leave blank
                'Basestoneminwt': "",  # Leave blank
                'Basestonemaxwt': "",  # Leave blank
                'Basemetalminwt': "",  # Leave blank
                'Basemetalmaxwt': "",  # Leave blank
                'Productiondeliverydate': "",  # Leave blank
                'Expecteddeliverydate': "",  # Leave blank
                'Blank_Column': "",  # Leave blank
                'SetPrice': "",  # Leave blank
                'StoneQuality': ""  # Leave blank
            }
            
            data.append(item_data)
            
            # Debug output for first few rows
            if index < 3:
                print(f"Row {index + 1} - Extracted values:")
                print(f"  SrNo: {sr_no}")
                print(f"  StyleCode: {style_code}")
                print(f"  ItemSize: {item_size}")
                print(f"  OrderQty: {order_qty}")
                print(f"  CustomerInstruction: {customer_instruction}")
                print(f"  Metal: {metal}")
                print(f"  Tone: {tone}")
                print(f"  SpecialRemarks: {special_remarks}")
                print("-" * 40)
        
        return data
        
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return []

# Main execution
if __name__ == "__main__":
    # Specify your Excel file path here
    excel_file_path = file  # Change this to your actual Excel file path
    
    print(f"Reading Excel file from: {excel_file_path}")
    
    # Process the Excel data
    processed_data = process_excel_data(excel_file_path)
    
    if processed_data:
        # Create DataFrame
        df_result = pd.DataFrame(processed_data)
        
        # Display the results
        print("\nProcessed DataFrame:")
        print("=" * 80)
        print(df_result.to_string(index=False))
        print("=" * 80)
        
        # Save to Excel file
        output_filename = 'Excel_Processed_Data1.xlsx'
        df_result.to_excel(output_filename, index=False)
        print(f"\nData successfully saved to '{output_filename}'")
        print(f"Total records processed: {len(df_result)}")
        print(f"Columns created: {len(df_result.columns)}")
        
        # Display column names
        print(f"\nColumn names:")
        for i, col in enumerate(df_result.columns, 1):
            print(f"{i:2d}. {col}")
        
        # Show first few rows
        print(f"\nFirst 3 rows of processed data:")
        print(df_result.head(3).to_string(index=False))
        
    else:
        print("No data was processed from the Excel file.")
    
    df_result


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1784680217.py, line 170)

In [18]:
from pathlib import Path
import pandas as pd

file = Path(r"C:\Users\Admin\Desktop\Aanya Inc\PO 6273.xls")
df = pd.read_excel(file)
df


,PURCHASE ORDER,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bill To:,Aanya Inc,NaN,NaN,NaN,Vendor,Jasani,NaN,NaN,NaN
3,NaN,"48W 48th Sreet, # 506",NaN,NaN,NaN,PO #,6273,NaN,NaN,NaN
4,NaN,"New York, NY 10036",NaN,NaN,NaN,Attn,Saurabh,Terms,NaN,NaN
5,NaN,Tele:- 212-278-0001,NaN,NaN,NaN,PO Date,2025-02-07 00:00:00,Ship Date,2025-03-28 00:00:00,NaN
6,NaN,Email:-dev@aanyainc.com,NaN,NaN,NaN,Metal Lock,2850,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Serial No,Style No,Picture,Description,Diamonds,NaN,Price,Qty,Amount,Sizes
9,1,QR0350H-I1,NaN,14KT \nWhite Gold\nMin-4.6gm,1.0 CT TW Round Shaped Diamond Band Size-7,Diamond 91 pcs & min 1cttw,600,330,198000,Size-7
